In [1]:
import torch
import torch.nn as nn
from fairseq.models.text_to_speech.vocoder import CodeHiFiGANVocoder
import soundfile as sf
from typing import List
import argparse
import logging
import json
from tqdm import tqdm
import os
import re
import traceback
from peft import PeftModel
from utils.speech2unit.speech2unit import Speech2Unit
import transformers
from transformers import AutoConfig, LlamaForCausalLM, LlamaTokenizer, GenerationConfig
import sys

2025-04-02 22:08:49.899123: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-02 22:08:49.919482: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-02 22:08:50.202029: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2025-04-02 22:08:50 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
/home/binhaoma/anaconda3/envs/SpeechGPT/lib/python3.8/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._registe

In [2]:
s2u_dir: str="utils/speech2unit/"

In [3]:
s2u = Speech2Unit(ckpt_dir=s2u_dir)

/home/binhaoma/anaconda3/envs/SpeechGPT/lib/python3.8/site-packages/fairseq/checkpoint_utils.py:315: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(f, map_

In [4]:
#original line 2
data = ["<581><63><991><470><821><167><462><104><125><491><405><503><853><487><319><914><476><534><74><635><908>" \
       "<52><270><382><268><412><260><323><534><485><948><86><539><757><41><740><585><611><916><726><584><819><80><658><941><9>" \
       "<127><184><341><465><27><418><6><649><131><567><232><807><6><646><345><88><64><6><33><262><823><64><739><335><739><242><610><712>"\
        "<581><665><662><393><145><920><781><191><513><589><331><529><500><683><200><734><105><561><445><333><259><398><813><260><323><321>"\
        "<323><692><521><757><858><498><338><907><351><706><53><208><555><557><403><295><579><728><284><333><885><790><754><515><526><436><340>"\
        "<611><423><673><764><689><555><764><497><375><588><377><824><829><637><859><750><3><382><245><997><588><237><201><674><763><619><118>"\
        "<174><682><588><523><506><544><197><802><376><651><119><965><377>"]

data


['<581><63><991><470><821><167><462><104><125><491><405><503><853><487><319><914><476><534><74><635><908><52><270><382><268><412><260><323><534><485><948><86><539><757><41><740><585><611><916><726><584><819><80><658><941><9><127><184><341><465><27><418><6><649><131><567><232><807><6><646><345><88><64><6><33><262><823><64><739><335><739><242><610><712><581><665><662><393><145><920><781><191><513><589><331><529><500><683><200><734><105><561><445><333><259><398><813><260><323><321><323><692><521><757><858><498><338><907><351><706><53><208><555><557><403><295><579><728><284><333><885><790><754><515><526><436><340><611><423><673><764><689><555><764><497><375><588><377><824><829><637><859><750><3><382><245><997><588><237><201><674><763><619><118><174><682><588><523><506><544><197><802><376><651><119><965><377>']

In [5]:
import re
# 提取所有 <数字> 中的数字
numbers = re.findall(r'<(\d+)>', data[0])
numbers = [int(num) for num in numbers]  # 转换为整数列表（可选）
print(numbers)
print(len(numbers))


[581, 63, 991, 470, 821, 167, 462, 104, 125, 491, 405, 503, 853, 487, 319, 914, 476, 534, 74, 635, 908, 52, 270, 382, 268, 412, 260, 323, 534, 485, 948, 86, 539, 757, 41, 740, 585, 611, 916, 726, 584, 819, 80, 658, 941, 9, 127, 184, 341, 465, 27, 418, 6, 649, 131, 567, 232, 807, 6, 646, 345, 88, 64, 6, 33, 262, 823, 64, 739, 335, 739, 242, 610, 712, 581, 665, 662, 393, 145, 920, 781, 191, 513, 589, 331, 529, 500, 683, 200, 734, 105, 561, 445, 333, 259, 398, 813, 260, 323, 321, 323, 692, 521, 757, 858, 498, 338, 907, 351, 706, 53, 208, 555, 557, 403, 295, 579, 728, 284, 333, 885, 790, 754, 515, 526, 436, 340, 611, 423, 673, 764, 689, 555, 764, 497, 375, 588, 377, 824, 829, 637, 859, 750, 3, 382, 245, 997, 588, 237, 201, 674, 763, 619, 118, 174, 682, 588, 523, 506, 544, 197, 802, 376, 651, 119, 965, 377]
167


In [6]:
import joblib
ckpt_dir = "./utils/speech2unit/"
km_path = os.path.join(ckpt_dir, "mhubert_base_vp_en_es_fr_it3_L11_km1000.bin")
class ApplyKmeans(object):
    def __init__(self, km_path):
        self.km_model = joblib.load(km_path)
        self.C_np = self.km_model.cluster_centers_.transpose()
        self.Cnorm_np = (self.C_np ** 2).sum(0, keepdims=True)

        self.C = torch.from_numpy(self.C_np)
        self.Cnorm = torch.from_numpy(self.Cnorm_np)
        if torch.cuda.is_available():
            self.C = self.C.cuda()
            self.Cnorm = self.Cnorm.cuda()

    def __call__(self, x):
        if isinstance(x, torch.Tensor):
            self.C = self.C.to(x)
            self.Cnorm = self.Cnorm.to(x)
            dist = (
                x.pow(2).sum(1, keepdim=True)
                - 2 * torch.matmul(x, self.C)
                + self.Cnorm
            )
            return -dist
        # dist.argmin(dim=1).cpu().numpy()
        else:
            dist = (
                (x ** 2).sum(1, keepdims=True)
                - 2 * np.matmul(x, self.C_np)
                + self.Cnorm_np
            )
            return -dist
        # np.argmin(dist, axis=1)
apply_kmeans = ApplyKmeans(km_path)

In [7]:
import os
import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import torch.optim as optim
import torchaudio
processed_parts = []
part = 'prompt/o1.wav'
if os.path.isfile(part.strip()) and os.path.splitext(part.strip())[-1] in [".wav", ".flac", ".mp4"]:
    processed_parts.append(s2u(part.strip(), merged=True))

# 我需要168   320 是一步
DEVICE = 'cuda'
waveform = s2u.feature_reader.read_audio(part).to(DEVICE)
ori_shape = waveform.shape[1]/320
# 初始 noise
noise_len = int((len(numbers) + 1) * 320)

# init_noise = torch.rand(1, noise_len, device=DEVICE)
# noise = torch.nn.Parameter(init_noise)
# optimizer = optim.Adam([noise], lr=5e-2)

T = waveform.shape[1]
noise_add = torch.nn.Parameter(torch.rand(1, noise_len, device=DEVICE))           # 用于叠加
optimizer = torch.optim.Adam([noise_add], lr=1e-2)



for step in range(10000):


    optimizer.zero_grad()
    budget = 0.02

    with torch.no_grad():
        noise_add.clamp_(-budget, budget)

    # 提取特征
    feat = s2u.feature_reader.get_feats(noise_add)


    # 聚类预测
    cluster_ids = apply_kmeans(feat)  # list[int]
    target = torch.tensor(numbers, dtype=torch.long, device=DEVICE)
    # cluster_ids = torch.tensor(cluster_ids).to('cuda')
    # one-hot 编码
    # num_classes = 1000
    # target_onehot = F.one_hot(target, num_classes=num_classes).float()  # shape: [B, 1000]


    # 用 MSE 来衡量 label 差异（因为没有 logits）
    loss = F.cross_entropy(cluster_ids, target.to('cuda'))
    # 也可以用 L1 Loss: F.l1_loss(pred, target)


    cluster_final_ids = cluster_ids.argmax(dim=1)  # 每行最大值的索引
    acc = (np.array(cluster_final_ids.cpu())==np.array(target.cpu())).sum()/len(target)

    loss.backward()
    optimizer.step()

    print(f'acc is {acc}')
    if acc == 1:
        torchaudio.save("prompt/output_noise.wav", noise_add.cpu(), sample_rate=16000)


    print(f"Step {step} | Loss: {loss.item():.4f}")





acc is 0.0
Step 0 | Loss: 62.5579
acc is 0.0
Step 1 | Loss: 56.8030
acc is 0.005988023952095809
Step 2 | Loss: 52.0376
acc is 0.011976047904191617
Step 3 | Loss: 42.5880
acc is 0.005988023952095809
Step 4 | Loss: 33.6500
acc is 0.005988023952095809
Step 5 | Loss: 29.8540
acc is 0.011976047904191617
Step 6 | Loss: 27.3316
acc is 0.017964071856287425
Step 7 | Loss: 25.6074
acc is 0.017964071856287425
Step 8 | Loss: 24.5431
acc is 0.023952095808383235
Step 9 | Loss: 23.4552
acc is 0.029940119760479042
Step 10 | Loss: 22.3216
acc is 0.03592814371257485
Step 11 | Loss: 21.3388
acc is 0.03592814371257485
Step 12 | Loss: 20.5098
acc is 0.041916167664670656
Step 13 | Loss: 19.7312
acc is 0.041916167664670656
Step 14 | Loss: 18.9646
acc is 0.04790419161676647
Step 15 | Loss: 18.1500
acc is 0.05389221556886228
Step 16 | Loss: 17.3168
acc is 0.0658682634730539
Step 17 | Loss: 16.5246
acc is 0.08383233532934131
Step 18 | Loss: 15.7374
acc is 0.10179640718562874
Step 19 | Loss: 14.9167
acc is 0.113

KeyboardInterrupt: 